In [ ]:
#| default_exp vanity

# Vanity addresses

> Use the notifyr bot to generate vanity addresses and get notified by DM when they are done.

## Run a simple generator that finds vanity addresses
Here is [an interesting article by Kris Constable about vanity addresses](https://krisconstable.com/generating-a-key-pair-with-nostr/). This is a great usecase for our notifyr decorator because calculating vanity addresses can be quite slow if we are looking for long words.

This simple module will let you look for an `npub` or a `hex` vanity address that starts with a particular pattern.

In [ ]:
#| export

from nostrfastr.nostr import PrivateKey
from nostrfastr.notifyr import notifyr

Lets make the function...

In [ ]:
#| export

def gen_vanity_pubkey(startswith: str, style='hex') -> PrivateKey:
    """randomly generate private keys until one matches the desire
    startswith for an npub or hex

    Parameters
    ----------
    startswith : str
        characters that the public key should start with. More chars
        means longer run time
    style : str, optional
        'npub' or 'hex' - npub is more commonly displayed on apps
        while hex is the true base private key with no encoding,
        by default 'hex'

    Returns
    -------
    PrivateKey
        returns a private key object
    """
    pubkey = ''
    if style == 'npub':
        npub_chars = '023456789acdefghjklmnpqrstuvwxyz'
        if not all(c in npub_chars for c in startswith):
            raise ValueError(f'character of selection not '
                              'in npub pattern ({npub_chars})')
        startswith = f'npub1{startswith}'
    else:
        hex_chars = 'abcdef0123456789'
        if not all(c in hex_chars for c in startswith):
            raise ValueError(f'character of selection not in '
                              'hex pattern ({hex_chars})')
    while not pubkey.startswith(startswith):
        privkey = PrivateKey()
        if style == 'npub':
            pubkey = privkey.public_key.bech32()
        else:
            pubkey = privkey.public_key.hex()
    return privkey

In [ ]:
from fastcore.test import test_fail

Make sure we don't allow characters that will never happen - test that these cases fail

In [ ]:
fail = lambda _: gen_vanity_pubkey(startswith='b', style='npub')
test_fail(fail)
fail = lambda _: gen_vanity_pubkey(startswith='g', style='hex')
test_fail(fail)


Generate a couple npubs!

In [ ]:
vanity_private_key_npub = gen_vanity_pubkey(startswith='23', style='npub')
vanity_private_key_hex = gen_vanity_pubkey(startswith='23', style='hex')

And make sure it worked...

In [ ]:
assert vanity_private_key_npub.public_key.bech32().startswith('npub123')
assert vanity_private_key_hex.public_key.hex().startswith('23')

Now we can also make a version of this that notifies you.

In [ ]:
#| export

vanity_notifyr = notifyr(gen_vanity_pubkey)

Remember that if you want a vanity notifyr that will go to a different address than the one you find in `vanity_notifyr.notifyr_privkey_hex` you can create your own notifyr like so:
```python
new_vanity_notifyr = notifyr(gen_vanity_pubkey, recipient_address=your_address)
```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()